In [33]:
import os
import sys

import seaborn as sns
#from ggplot import *
import pandas as pd
import numpy as np
#import mxnet as mx 
import seaborn as sns
#from ggplot import *
import pandas as pd
import numpy as np
from sklearn.metrics import roc_auc_score, f1_score
#from matplotlib import pyplot as plt
#from sklearn.feature_selection import f_classif
from sklearn.model_selection import train_test_split
#from sklearn.manifold import TSNE
from sklearn.ensemble import RandomForestClassifier
#from sklearn.preprocessing import StandardScaler
import logging
from sklearn.linear_model import LogisticRegression
#import xgboost as xgb

In [2]:
aisles = pd.read_csv('aisles.csv')
departments = pd.read_csv('departments.csv')
order_products_prior = pd.read_csv('order_products__prior.csv')
order_products_train = pd.read_csv('order_products__train.csv')

orders = pd.read_csv('orders.csv')
products = pd.read_csv('products.csv')
sample_submission = pd.read_csv('sample_submission.csv')

In [143]:
order_products_prior[0:16217244]

,order_id,product_id,add_to_cart_order,reordered
0,2,33120,1,1
1,2,28985,2,1
2,2,9327,3,0
3,2,45918,4,1
4,2,30035,5,0
5,2,17794,6,1
6,2,40141,7,1
7,2,1819,8,1
8,2,43668,9,0
9,3,33754,1,1


In [146]:
#order_products_prior[0:10811496].to_csv('order_products_prior1.csv')
order_products_prior[10811497:21622992].to_csv('order_products_prior2.csv')
order_products_prior[21622993:32434489].to_csv('order_products_prior3.csv')


In [ ]:
# First convert days_since_prior_order to days
#orders

In [3]:
orders['days_since_prior_order'] = orders['days_since_prior_order'].fillna(0)

In [4]:
day_list = []
for i in range(len(orders['days_since_prior_order'])):
    if i==0:
        is_first = 1
        tot_days = 0
        day_list.append(tot_days)
        prev_user_id = orders['user_id'][i]
    else:
        if orders['user_id'][i] == prev_user_id:
            tot_days = tot_days + orders['days_since_prior_order'][i]
            day_list.append(tot_days)
        else:
            tot_days=0
            prev_user_id = orders['user_id'][i]
            day_list.append(tot_days)
    
orders['days'] = np.array(day_list)    

In [5]:
orders.to_csv('order_days.csv')

In [6]:
order_products_prior_all = pd.merge(order_products_prior, orders.drop(['eval_set'],axis=1), how='left', on='order_id')
order_products_train_all = pd.merge(order_products_train, orders.drop(['eval_set'],axis=1), how='left', on='order_id')

In [7]:
order_products_train_all['target'] = 1

In [8]:
order_products_prior_all_target = pd.merge(order_products_prior_all, order_products_train_all[['user_id', 'product_id', 'target']], how='left', on=['user_id', 'product_id'])


In [9]:
orders_train = orders.loc[np.asarray(orders['eval_set'] == 'train') | np.asarray(orders['eval_set'] == 'test')]
target = order_products_prior_all_target[['product_id', 'user_id', 'target']].drop_duplicates()
target['target'] = target['target'].fillna(0)
target_more = pd.merge(target, orders_train[['user_id', 'eval_set']], how='left', on='user_id')

In [10]:
#order_products_prior_all_target = order_products_prior_all_target.drop(['target'],axis=1)

In [11]:
order_products_prior_train_order = pd.merge(order_products_prior_all_target, orders_train.drop(['eval_set'], axis=1), how='left', on='user_id', suffixes = ('_prior', '_train'))
#order_products_prior_train_order

In [19]:
features = order_products_prior_train_order[['product_id', 'user_id']]
features['days_diff'] = order_products_prior_train_order['days_prior'].subtract(order_products_prior_train_order['days_train'])
features

/Applications/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


,product_id,user_id,days_diff
0,33120,202279,-153.0
1,28985,202279,-153.0
2,9327,202279,-153.0
3,45918,202279,-153.0
4,30035,202279,-153.0
5,17794,202279,-153.0
6,40141,202279,-153.0
7,1819,202279,-153.0
8,43668,202279,-153.0
9,33754,205970,-72.0


In [20]:
features['order_hour_of_day_diff'] = order_products_prior_train_order['order_hour_of_day_prior'].subtract(order_products_prior_train_order['order_hour_of_day_train'])
features['order_dow_diff'] = order_products_prior_train_order['order_dow_prior'].subtract(order_products_prior_train_order['order_dow_train'])
#features['order_hour_of_day_diff'] = order_products_prior_train_order['order_hour_of_day_prior'] - order_products_prior_train_order['order_hour_of_day_train']
features

/Applications/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/Applications/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


,product_id,user_id,days_diff,order_hour_of_day_diff,order_dow_diff
0,33120,202279,-153.0,-5,4
1,28985,202279,-153.0,-5,4
2,9327,202279,-153.0,-5,4
3,45918,202279,-153.0,-5,4
4,30035,202279,-153.0,-5,4
5,17794,202279,-153.0,-5,4
6,40141,202279,-153.0,-5,4
7,1819,202279,-153.0,-5,4
8,43668,202279,-153.0,-5,4
9,33754,205970,-72.0,-2,5


In [21]:
features_means = features.groupby(['product_id','user_id'], as_index=False).agg(np.mean)
features_max = features.groupby(['product_id','user_id'], as_index=False).agg(np.amax)
features_min = features.groupby(['product_id','user_id'], as_index=False).agg(np.amin)
features_median = features.groupby(['product_id','user_id'], as_index=False).agg(np.median)
features_var = features.groupby(['product_id','user_id'], as_index=False).agg(np.var)


In [22]:
features_means

,product_id,user_id,days_diff,order_hour_of_day_diff,order_dow_diff
0,1,138,-42.000000,2.000000,0.000000
1,1,709,-28.000000,-1.000000,0.000000
2,1,764,-16.000000,5.000000,1.500000
3,1,777,-98.000000,-5.000000,0.000000
4,1,825,-142.000000,-5.000000,2.000000
5,1,910,0.000000,-1.000000,0.000000
6,1,1052,-80.500000,-2.000000,0.000000
7,1,1379,-159.000000,0.000000,2.000000
8,1,1480,-57.000000,-0.333333,-1.000000
9,1,1494,-81.333333,0.666667,0.333333


In [23]:
features_all = features_means.copy()
features_all.columns = ['product_id','user_id', 'days_diff_mean', 'order_hour_of_day_diff_mean', 'order_dow_diff_mean']

features_all[['days_diff_max', 'order_hour_of_day_diff_max', 'order_dow_diff_max']] = features_max[['days_diff', 'order_hour_of_day_diff', 'order_dow_diff']]
features_all[['days_diff_min', 'order_hour_of_day_diff_min', 'order_dow_diff_min']] = features_min[['days_diff', 'order_hour_of_day_diff', 'order_dow_diff']]
features_all[['days_diff_median', 'order_hour_of_day_diff_median', 'order_dow_diff_median']] = features_median[['days_diff', 'order_hour_of_day_diff', 'order_dow_diff']]
features_all[['days_diff_var', 'order_hour_of_day_diff_var', 'order_dow_diff_var']] = features_var[['days_diff', 'order_hour_of_day_diff', 'order_dow_diff']]
features_all

,product_id,user_id,days_diff_mean,order_hour_of_day_diff_mean,order_dow_diff_mean,days_diff_max,order_hour_of_day_diff_max,order_dow_diff_max,days_diff_min,order_hour_of_day_diff_min,order_dow_diff_min,days_diff_median,order_hour_of_day_diff_median,order_dow_diff_median,days_diff_var,order_hour_of_day_diff_var,order_dow_diff_var
0,1,138,-42.000000,2.000000,0.000000,-21.0,5,0,-63.0,-1,0,-42.0,2.0,0.0,882.000000,18.000000,0.000000
1,1,709,-28.000000,-1.000000,0.000000,-28.0,-1,0,-28.0,-1,0,-28.0,-1.0,0.0,NaN,NaN,NaN
2,1,764,-16.000000,5.000000,1.500000,-5.0,7,2,-27.0,3,1,-16.0,5.0,1.5,242.000000,8.000000,0.500000
3,1,777,-98.000000,-5.000000,0.000000,-98.0,-5,0,-98.0,-5,0,-98.0,-5.0,0.0,NaN,NaN,NaN
4,1,825,-142.000000,-5.000000,2.000000,-142.0,-5,2,-142.0,-5,2,-142.0,-5.0,2.0,NaN,NaN,NaN
5,1,910,0.000000,-1.000000,0.000000,0.0,-1,0,0.0,-1,0,0.0,-1.0,0.0,NaN,NaN,NaN
6,1,1052,-80.500000,-2.000000,0.000000,-56.0,2,0,-105.0,-6,0,-80.5,-2.0,0.0,1200.500000,32.000000,0.000000
7,1,1379,-159.000000,0.000000,2.000000,-159.0,0,2,-159.0,0,2,-159.0,0.0,2.0,NaN,NaN,NaN
8,1,1480,-57.000000,-0.333333,-1.000000,-44.0,0,0,-71.0,-1,-2,-56.0,0.0,-1.0,183.000000,0.333333,1.000000
9,1,1494,-81.333333,0.666667,0.333333,-68.0,2,2,-91.0,0,-1,-85.0,0.0,0.0,142.333333,1.333333,2.333333


In [27]:
final_data = pd.merge(features_all, target_more, how='left', on=['product_id', 'user_id'])

In [28]:
final_data_train = final_data.loc[final_data['eval_set']=='train']
final_data_train

,product_id,user_id,days_diff_mean,order_hour_of_day_diff_mean,order_dow_diff_mean,days_diff_max,order_hour_of_day_diff_max,order_dow_diff_max,days_diff_min,order_hour_of_day_diff_min,order_dow_diff_min,days_diff_median,order_hour_of_day_diff_median,order_dow_diff_median,days_diff_var,order_hour_of_day_diff_var,order_dow_diff_var,target,eval_set
0,1,138,-42.000000,2.000000,0.000000,-21.0,5,0,-63.0,-1,0,-42.0,2.0,0.0,882.000000,18.000000,0.000000,0.0,train
1,1,709,-28.000000,-1.000000,0.000000,-28.0,-1,0,-28.0,-1,0,-28.0,-1.0,0.0,NaN,NaN,NaN,0.0,train
3,1,777,-98.000000,-5.000000,0.000000,-98.0,-5,0,-98.0,-5,0,-98.0,-5.0,0.0,NaN,NaN,NaN,0.0,train
6,1,1052,-80.500000,-2.000000,0.000000,-56.0,2,0,-105.0,-6,0,-80.5,-2.0,0.0,1200.500000,32.000000,0.000000,0.0,train
9,1,1494,-81.333333,0.666667,0.333333,-68.0,2,2,-91.0,0,-1,-85.0,0.0,0.0,142.333333,1.333333,2.333333,0.0,train
10,1,1540,-81.294118,13.529412,0.235294,-8.0,22,5,-167.0,-1,-1,-71.0,19.0,0.0,2695.970588,85.389706,2.566176,1.0,train
13,1,2833,-128.000000,-3.000000,-1.000000,-128.0,-3,-1,-128.0,-3,-1,-128.0,-3.0,-1.0,NaN,NaN,NaN,0.0,train
14,1,2850,-157.000000,9.000000,1.000000,-157.0,9,1,-157.0,9,1,-157.0,9.0,1.0,NaN,NaN,NaN,0.0,train
16,1,3010,-61.000000,-4.000000,2.000000,-61.0,-4,2,-61.0,-4,2,-61.0,-4.0,2.0,NaN,NaN,NaN,0.0,train
19,1,3904,-69.000000,-9.000000,1.000000,-69.0,-9,1,-69.0,-9,1,-69.0,-9.0,1.0,NaN,NaN,NaN,0.0,train


In [77]:
y = final_data_train['target']
X = final_data_train.drop(['target', 'eval_set'], axis=1)
X = X.fillna(0)


In [80]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)


In [81]:
prod_users_train = X_train[['product_id', 'user_id']]
X_train = X_train.drop(['product_id', 'user_id'], axis=1)
prod_users_test = X_test[['product_id', 'user_id']]
X_test = X_test.drop(['product_id', 'user_id'], axis=1)


In [130]:
final_data_test = final_data.loc[final_data['eval_set']=='test']
final_data_test

,product_id,user_id,days_diff_mean,order_hour_of_day_diff_mean,order_dow_diff_mean,days_diff_max,order_hour_of_day_diff_max,order_dow_diff_max,days_diff_min,order_hour_of_day_diff_min,order_dow_diff_min,days_diff_median,order_hour_of_day_diff_median,order_dow_diff_median,days_diff_var,order_hour_of_day_diff_var,order_dow_diff_var,target,eval_set
2,1,764,-16.000000,5.000000,1.500000,-5.0,7,2,-27.0,3,1,-16.0,5.0,1.5,242.000000,8.000000,0.500000,0.0,test
4,1,825,-142.000000,-5.000000,2.000000,-142.0,-5,2,-142.0,-5,2,-142.0,-5.0,2.0,NaN,NaN,NaN,0.0,test
5,1,910,0.000000,-1.000000,0.000000,0.0,-1,0,0.0,-1,0,0.0,-1.0,0.0,NaN,NaN,NaN,0.0,test
7,1,1379,-159.000000,0.000000,2.000000,-159.0,0,2,-159.0,0,2,-159.0,0.0,2.0,NaN,NaN,NaN,0.0,test
8,1,1480,-57.000000,-0.333333,-1.000000,-44.0,0,0,-71.0,-1,-2,-56.0,0.0,-1.0,183.000000,0.333333,1.000000,0.0,test
11,1,1598,-20.333333,5.000000,0.666667,-12.0,9,2,-35.0,2,0,-14.0,4.0,0.0,162.333333,13.000000,1.333333,0.0,test
12,1,1647,-134.200000,3.600000,0.200000,-57.0,8,3,-222.0,0,-1,-136.0,4.0,0.0,4495.200000,12.800000,2.700000,0.0,test
15,1,2857,-25.000000,-13.000000,-4.000000,-25.0,-13,-4,-25.0,-13,-4,-25.0,-13.0,-4.0,NaN,NaN,NaN,0.0,test
17,1,3029,-129.000000,6.000000,-1.000000,-129.0,6,-1,-129.0,6,-1,-129.0,6.0,-1.0,NaN,NaN,NaN,0.0,test
18,1,3395,-65.000000,-5.000000,-2.000000,-65.0,-5,-2,-65.0,-5,-2,-65.0,-5.0,-2.0,NaN,NaN,NaN,0.0,test


In [70]:
rf = RandomForestClassifier(n_estimators=10, criterion='gini', max_depth=None, min_samples_split=200, min_samples_leaf=1,
                            min_weight_fraction_leaf=0.0, max_features='auto', max_leaf_nodes=None, 
                            min_impurity_split=1e-07, bootstrap=True, oob_score=False, n_jobs=1, random_state=None, 
                            verbose=0, warm_start=False, class_weight=None)

rf.fit(X_train, y_train)


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=200, min_weight_fraction_leaf=0.0,
            n_estimators=10, n_jobs=1, oob_score=False, random_state=None,
            verbose=0, warm_start=False)

In [71]:
#pred_train_prob = rf.predict_proba(X_train)
#pred_train = rf.predict(X_train)
pred_test_prob = rf.predict_proba(X_test)
pred_test = rf.predict(X_test)
#print('train auc', roc_auc_score(y_train, pred_train_prob))
#print('test f1', f1_score(y_test, pred_test))

#f1_score 


In [72]:
print('train auc', roc_auc_score(y_test, pred_test_prob[:,1]))
print('test f1', f1_score(y_test, pred_test))


train auc 0.797915329946
test f1 0.125943169482


In [131]:
f1_df = prod_users_test.copy()
f1_df['y_test'] = y_test
f1_df['pred_test'] = pred_test_prob[:,1]
cutoff = 0.35
f1_df_subset = f1_df.loc[np.asarray(f1_df['y_test']==1) | np.asarray(f1_df['pred_test']>cutoff)]




In [132]:
f1_df_subset['pred_test'] = f1_df_subset['pred_test']>cutoff
#f1_df_subset

/Applications/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [133]:
f1_df_group = f1_df_subset.groupby(['product_id', 'user_id'])

f1_tot = 0
n=0
for name, groups in f1_df_group:
    #print(f1_score(groups['y_test'], groups['pred_test']))
    f1_tot = f1_tot+f1_score(groups['y_test'], groups['pred_test'])
    n=n+1
    
print('Mean f1_score is', f1_tot/n)

/Applications/anaconda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/Applications/anaconda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1115: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true samples.
  'recall', 'true', average, warn_for)


Mean f1_score is 0.208222571016


In [134]:
nones = len(np.unique(f1_df['user_id'])) - len(np.unique(f1_df_subset['user_id']))
mean_f1 = (f1_tot + nones)/(nones + n)
print(mean_f1)

0.329768565274


In [96]:
0.239754354602
0.4: 0.316012943917
0.3: 0.331721493897
#f1_tot

11659.0

## Significant Findings
Overall event rate = 13.1%

In [26]:
target_more

,product_id,user_id,target,eval_set
0,33120,202279,1.0,train
1,28985,202279,0.0,train
2,9327,202279,0.0,train
3,45918,202279,0.0,train
4,30035,202279,0.0,train
5,17794,202279,0.0,train
6,40141,202279,0.0,train
7,1819,202279,0.0,train
8,43668,202279,0.0,train
9,33754,205970,1.0,train
